# Cluster Interaction Basics
> Intro to Seldon Deployment CRD!

- toc: true 
- badges: true
- comments: true
- categories: [kubernetes, docker]
- image: images/chart-preview.png

## Intro
In this post we will walk through the basics behind the seldon custom resource definition. 


In [ ]:
!kind create cluster 

When working in kubernetes, I like to create a new namespace for that work. For those familiar with python, namespaces are similar to using virtual environments for different projects. Create a namespace with this command: 

In [ ]:
!kubectl create namespace seldon-intro

Then use kubens to set this as your current namespace:

In [ ]:
!kubens seldon-intro

### Install Seldon
I suggest using [helm]() to install seldon. If you haven't used helm before, use [this page](https://helm.sh/) to find install instructions. (If you're on a mac, just use `brew install helm`). For those familiar with python, helm is like the pip for kubernetes. It works by using [helm charts](https://helm.sh/docs/topics/charts/). A chart is a group of files that describe a higher level application using built-in kubernetes resources. For example, you could use a helm chart to deploy a web application. You can learn more about them [here](https://helm.sh/docs/topics/charts/).

Once helm is installed and you have your kubernetes cluster up and running (refer to [part 1] of this series if you don't have a cluster running yet), use helm to install seldon-core and seldon-core-operator. 

In [ ]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon-intro

The docker image name used to create the seldon-controller-manager pod and deployment is called `docker.io/seldonio/seldon-core-operator:1.2.1`. After running the previous helm install, you will see the resource `seldon-controller-manager` created as a deployment, and a pod with that name included. 

In [13]:
!kubectl get pods -o wide

NAME                                        READY   STATUS    RESTARTS   AGE   IP           NODE                 NOMINATED NODE   READINESS GATES
seldon-controller-manager-769694875-gk4gl   1/1     Running   0          18m   10.244.0.5   kind-control-plane   <none>           <none>


In [12]:
!kubectl get deployments

NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
seldon-controller-manager   1/1     1            1           18m


Seldon-controller-manager is a kubernetes operator. As described [here](https://www.jaegertracing.io/docs/1.16/operator/), kubernetes operators are extensions of the kubernetes api that allow users to easily package and deploy complex applications on kubernetes. As you will see throughout the rest of this series. The seldon-core-operator adds a lot of functionality to the kubernetes cluster and allows us to interact with seldon deployments as if they were a built-in kubernetes object. 

As described in the kubernetes docs [here](https://kubernetes.io/docs/concepts/extend-kubernetes/operator/), operators follow the controller pattern, which means they are responsible for keep the desired state of the custom resource they are responsible for. 
The seldon-core-operator is responsbile for the Seldon Deployment custom resource. That means, when we create or edit a seldon deployment, the seldon-core-operator is responsible for adjusting the kubernetes deployment to the desired state to meet the new edits applied by the user. 


In [27]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris-model
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: predictor
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/iris-model configured


In [28]:
!kubectl get pods

NAME                                                              READY   STATUS    RESTARTS   AGE
iris-model-predictor-0-sklearn-iris-classifier-5f5b59678-8bpwk    0/2     Running   0          3s
iris-model-sklearn-iris-predictor-0-sklearn-iris-classifie4l84z   0/2     Running   0          17s
seldon-controller-manager-769694875-gk4gl                         1/1     Running   0          30m
